In [1]:
from sklearn.metrics import ndcg_score

import pandas as pd
import numpy as np

In [151]:
# download the dataset from here:
#https://www.microsoft.com/en-us/research/project/mslr/

train = pd.read_csv("train_sub_fold1.txt", header=None, sep=" ")

### libSVM format
read about this file format here: https://catboost.ai/docs/concepts/input-data_libsvm.html

In [152]:
def extract_query(qstr):
    return qstr[4:]

def extract_features(features):
    return features.split(':')[1]

In [153]:
def df_preparing(df):
    df[1] = df[1].apply(extract_query)
    df[df.columns[2:-1]] = df[df.columns[2:-1]].applymap(extract_features)
    df = df.drop(138, axis=1)
    return df

In [154]:
train_df = df_preparing(train)

In [155]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2,1,3,3,0,0,3,1,1,0,...,62,11089534,2,116,64034,13,3,0,0,0
1,2,1,3,0,3,0,3,1,0,1,...,54,11089534,2,124,64034,1,2,0,0,0
2,0,1,3,0,2,0,3,1,0,0.666667,...,45,3,1,124,3344,14,67,0,0,0
3,2,1,3,0,3,0,3,1,0,1,...,56,11089534,13,123,63933,1,3,0,0,0
4,1,1,3,0,3,0,3,1,0,1,...,64,5,7,256,49697,1,13,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723407,0,29992,2,0,1,1,2,1,0,0.50000,...,39,7,2,13829,35302,21,8,0,0,0
723408,0,29992,2,1,1,1,2,1,0.50000,0.50000,...,28,0,0,26074,35101,14,7,1,0,0
723409,0,29992,2,2,2,2,2,1,1,1,...,58,11,2,2995,62170,4,8,0,0,0
723410,0,29992,2,0,0,0,2,1,0,0,...,65,0,0,138,56419,4,3,0,0,0


In [9]:
# point-wise
import sklearn
from sklearn.linear_model import LinearRegression

X = train_df[train_df.columns[2:]]
y = train_df[0]
reg = LinearRegression().fit(X, y)


# printing R^2
reg.score(X,y)

0.13349313112592154

In [10]:
## Write code segment to  read the 'test' libsvm file from
# https://www.microsoft.com/en-us/research/project/mslr/
# Then:
# 1) use the above reg object to predict all scores for all samples.
# 2) calculate average NDCG for all quesries in the test file

Week-10+11 .... Slide 17 === Use multiclass SVMs to learn to predict the labels for query document pairs 


### Write code segment to  read the 'test' libsvm file from

In [11]:
test = pd.read_csv("test_IR_excercise.txt", header=None, sep=" ")

In [147]:
test_df = df_preparing(test)
test_df

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2,13,2,0,2,1,2,1,0,1,...,35,1,0,266,25070,28,7,0,0,0
1,1,13,2,0,0,0,2,1,0,0,...,17,93,0,153,12860,65,158,0,0,0
2,3,13,2,0,1,0,2,1,0,0.50000,...,19,0,0,153,1131,112,141,0,0,0
3,1,13,2,0,2,1,2,1,0,1,...,50,81775,0,560,61224,1,14,0,0,0
4,0,13,1,0,0,0,1,0.50000,0,0,...,24,0,0,57953,15600,15,12,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241516,1,29998,3,0,1,1,3,1,0,0.333333,...,60,9,1,550,49786,23,102,0,0,0
241517,1,29998,3,1,0,0,3,1,0.333333,0,...,91,268,0,181,53904,1,2,0,0,0
241518,1,29998,3,0,3,0,3,1,0,1,...,69,14,9,4591,32817,3,9,0,0,0
241519,0,29998,3,0,1,1,3,1,0,0.333333,...,51,0,1,1549,59332,10,4,0,0,0


In [13]:
X_test = test_df[test_df.columns[2:]] 
y_test = test_df[0] 

### 1) use the above reg object to predict all scores for all samples.

In [14]:
y_pred = reg.predict(X_test)
y_pred

array([0.76923034, 0.29349399, 0.3523804 , ..., 0.80657282, 0.43872805,
       0.19482099])

### 2) calculate average NDCG for all queries in the test file

In [118]:
def print_type_shape(x):
    print(type(x))
    print(x.shape)

In [127]:
print_type_shape(y_pred)

<class 'numpy.ndarray'>
(241521,)


In [16]:
y_pred_reshaped = y_pred.reshape(1, y_pred.shape[0])

In [128]:
print_type_shape(y_pred_reshaped)

<class 'numpy.ndarray'>
(1, 241521)


In [129]:
print_type_shape(y_test)

<class 'pandas.core.series.Series'>
(241521,)


In [19]:
y_test_reshaped = y_test.values.reshape(1, y_pred.shape[0])

In [130]:
print_type_shape(y_test_reshaped)
y_test_reshaped

<class 'numpy.ndarray'>
(1, 241521)


array([[2, 1, 3, ..., 1, 0, 0]], dtype=int64)

In [21]:
ndcg_score(y_test_reshaped, y_pred_reshaped)

0.9258607285957392

In [22]:
#!pip install xgboost
import xgboost as xgb

In [23]:
training_data = xgb.DMatrix('train_sub_fold1.txt')
#DMatrix is an internal data structure that is used by XGBoost, 
#which is optimized for both memory efficiency and training speed.

# 3) use the set_group([]) method of DMatrix to specify the number of urls(items) for each query. The input of this method should
# be an an array

In [162]:
cols = train_df.columns
sdfg = training_data.set_group(cols)
print(type(sdfg))
print(sdfg)

<class 'NoneType'>
None


In [136]:
training_data
print(f'Number of rows: {training_data.num_row()}')
print(f'Label: {training_data.feature_names}')
#training_data.data()

Number of rows: 723412
Label: None


In [29]:
testing_data = xgb.DMatrix('test_IR_excercise.txt')
testing_data

In [131]:
print(f'Number of rows: {testing_data.num_row()}')
print(f'Label: {testing_data.feature_names}')

Number of rows: 241521
Label: None


In [39]:
# list-wise
ltr_lambdamart_param = [('objective','rank:ndcg'),('max_depth',3), ('eta',0.1), ('seed',404)]

lambdamart_model = xgb.train(ltr_lambdamart_param, training_data)


In [46]:
pred_lambdamart = lambdamart_model.predict(testing_data)

In [48]:
pred_lambdamart

array([0.50383204, 0.27758095, 0.27758095, ..., 0.5274218 , 0.42350563,
       0.5274218 ], dtype=float32)

In [55]:
#lambdarank (pairwise LTR)
ltr_lambdarank_param = [('objective','rank:pairwise'),('max_depth',3), ('eta',0.1), ('seed',404)]

 
lambdarank_model = xgb.train(ltr_lambdarank_param, training_data)


In [117]:
pred_lambdarank = lambdarank_model.predict(testing_data)
pred_lambdarank

array([0.51874346, 0.4311399 , 0.4311399 , ..., 0.6515914 , 0.4926514 ,
       0.4907252 ], dtype=float32)

In [57]:
test = pd.read_csv("test_IR_excercise.txt", header=None, sep=" ")


In [92]:
test

,0,1,2,3,4,5,6,7,8,9,...,129,130,131,132,133,134,135,136,137,138
0,2,qid:13,1:2,2:0,3:2,4:1,5:2,6:1,7:0,8:1,...,128:1,129:0,130:266,131:25070,132:28,133:7,134:0,135:0,136:0,NaN
1,1,qid:13,1:2,2:0,3:0,4:0,5:2,6:1,7:0,8:0,...,128:93,129:0,130:153,131:12860,132:65,133:158,134:0,135:0,136:0,NaN
2,3,qid:13,1:2,2:0,3:1,4:0,5:2,6:1,7:0,8:0.50000,...,128:0,129:0,130:153,131:1131,132:112,133:141,134:0,135:0,136:0,NaN
3,1,qid:13,1:2,2:0,3:2,4:1,5:2,6:1,7:0,8:1,...,128:81775,129:0,130:560,131:61224,132:1,133:14,134:0,135:0,136:0,NaN
4,0,qid:13,1:1,2:0,3:0,4:0,5:1,6:0.50000,7:0,8:0,...,128:0,129:0,130:57953,131:15600,132:15,133:12,134:0,135:0,136:0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241516,1,qid:29998,1:3,2:0,3:1,4:1,5:3,6:1,7:0,8:0.333333,...,128:9,129:1,130:550,131:49786,132:23,133:102,134:0,135:0,136:0,NaN
241517,1,qid:29998,1:3,2:1,3:0,4:0,5:3,6:1,7:0.333333,8:0,...,128:268,129:0,130:181,131:53904,132:1,133:2,134:0,135:0,136:0,NaN
241518,1,qid:29998,1:3,2:0,3:3,4:0,5:3,6:1,7:0,8:1,...,128:14,129:9,130:4591,131:32817,132:3,133:9,134:0,135:0,136:0,NaN
241519,0,qid:29998,1:3,2:0,3:1,4:1,5:3,6:1,7:0,8:0.333333,...,128:0,129:1,130:1549,131:59332,132:10,133:4,134:0,135:0,136:0,NaN


### 4) calculate average NDCG for both lambdarank_model and lambdamart_model

#### Reshape pred_lambdamart

In [119]:
print_type_shape(pred_lambdamart)

<class 'numpy.ndarray'>
(241521,)


In [104]:
pred_lambdamart_reshaped = pred_lambdamart.reshape(1, pred_lambdamart.shape[0])

In [120]:
print_type_shape(pred_lambdamart_reshaped)

<class 'numpy.ndarray'>
(1, 241521)


#### Reshape pred_lambdarank

In [121]:
print_type_shape(pred_lambdarank)

<class 'numpy.ndarray'>
(241521,)


In [114]:
pred_lambdarank_reshaped = pred_lambdarank.reshape(1, pred_lambdarank.shape[0])

In [123]:
print_type_shape(pred_lambdarank_reshaped)

<class 'numpy.ndarray'>
(1, 241521)


#### Reshape y_test

In [106]:
y_test_xgb = test[0]

In [124]:
print_type_shape(y_test_xgb)

<class 'pandas.core.series.Series'>
(241521,)


In [108]:
y_test_xgb_reshaped = y_test_xgb.values.reshape(1, y_test_xgb.shape[0])

In [125]:
print_type_shape(y_test_xgb_reshaped)

<class 'numpy.ndarray'>
(1, 241521)


#### NDCG score

In [110]:
ndcg_score(y_test_xgb_reshaped, pred_lambdamart_reshaped)

0.9179210315408598

In [116]:
ndcg_score(y_test_xgb_reshaped, pred_lambdarank_reshaped)

0.9240950704313468